- Realizamos una pequeña muestra con el código del alquiler

In [1]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from datetime import datetime
import uuid

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)  # Añadir pausa después de aceptar cookies

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles
max_inmuebles = 50  # Número máximo de inmuebles que queremos recolectar

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble():
    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Descripción del inmueble (H1)
            descripcion = soup_inmueble.find('h1').text.strip() if soup_inmueble.find('h1') else "Descripción no disponible"

            # Localización en el primer <p> después del <h1> (ejemplo: Centro (Galapagar))
            localizacion = soup_inmueble.find('p').text.strip() if soup_inmueble.find('p') else "Localización no disponible"

            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Agregar el timestamp de scrapeo y un identificador único
            data = {
                'Descripción': descripcion,
                'Localización': localizacion,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler',
                'timestamp_scrapeo': datetime.now().isoformat(),  # Timestamp de scrapeo
                'id': str(uuid.uuid4())  # ID único para cada inmueble
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Función para avanzar al siguiente inmueble
def ir_al_siguiente_inmueble():
    try:
        siguiente_boton = WebDriverWait(browser, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.navigation__link.navigation__link--next'))
        )
        siguiente_boton.click()
        time.sleep(2)  # Pausa para cargar el siguiente inmueble
        return True
    except Exception as e:
        print(f"No se encontró el botón de 'Siguiente': {e}")
        return False

# Iniciar en el primer inmueble
try:
    primer_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.ad-preview'))
    )
    primer_inmueble.click()
    time.sleep(2)  # Pausa para cargar la página del primer inmueble
except Exception as e:
    print(f"Error al abrir el primer inmueble: {e}")

# Bucle para extraer datos de cada inmueble hasta alcanzar el máximo o agotar los inmuebles
while len(all_data) < max_inmuebles:
    data = extraer_datos_inmueble()
    if data:
        all_data.append(data)
        print(f"Inmueble {inmueble_counter}: {data}")
        inmueble_counter += 1

    # Intentar ir al siguiente inmueble
    if not ir_al_siguiente_inmueble():
        break  # Si no hay más inmuebles, salir del bucle

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('muestra_alquiler_50_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Descripción': 'Piso en alquiler en Zona Avenida de Europa', 'Localización': 'Zona Avenida de Europa (Pozuelo de Alarcón)', 'Precio': '1.400', 'Superficie Construida': '65', 'Última Actualización': 'Anuncio actualizado el 30/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '65 m²'), ('Habitaciones:', '1'), ('Baños:', '1'), ('Planta:', '2ª'), ('Referencia:', 'IF5032-avda-1d-V T 40m')], 'Tipo de operación': 'Alquiler', 'timestamp_scrapeo': '2024-10-30T18:16:48.188293', 'id': '203bb15f-240e-46ce-bb39-9d172cdb8096'}
Inmueble 2: {'Descripción': 'Piso en alquiler en Calle Gran Vía, cerca de Calle de García Molinas', 'Localización': 'Universidad-Malasaña (Distrito Centro. Madrid Capital)', 'Precio': '3.500', 'Superficie Construida': '122', 'Última Actualización': 'Anuncio actualizado el 03/10/2024', 'Consumo Energético': 'Consumo:70 kWh/m² año', 'Emisiones CO2': 'Emisiones:15 Kg CO₂/m² año', 'Características': [('Superf

In [2]:
df

,Descripción,Localización,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación,timestamp_scrapeo,id
0,Piso en alquiler en Zona Avenida de Europa,Zona Avenida de Europa (Pozuelo de Alarcón),1.400,65,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 65 m²), (Habitacione...",Alquiler,2024-10-30T18:16:48.188293,203bb15f-240e-46ce-bb39-9d172cdb8096
1,"Piso en alquiler en Calle Gran Vía, cerca de C...",Universidad-Malasaña (Distrito Centro. Madrid ...,3.500,122,Anuncio actualizado el 03/10/2024,Consumo:70 kWh/m² año,Emisiones:15 Kg CO₂/m² año,"[(Superficie construida:, 122 m²), (Superficie...",Alquiler,2024-10-30T18:17:01.098050,c4bc1e39-dc09-42e9-924e-8e7edc2c5603
2,Ático en alquiler en Paseo de la Tierra de Mel...,Las Tablas (Distrito Fuencarral-El Pardo. Madr...,3.500,350,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 350 m²), (Superficie...",Alquiler,2024-10-30T18:17:09.123565,bdc8cf41-bb6f-4794-aea5-6c5b33ab3e08
3,Casa unifamiliar en alquiler en Calle XIX,Las Matas-Los Peñascales (Las Rozas de Madrid),4.000,600,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 600 m²), (Superficie...",Alquiler,2024-10-30T18:17:17.227102,9c2eb3a8-67e7-454e-a57c-6bff6b49b2fb
4,Casa adosada en alquiler en Sector B,Sector B (Boadilla del Monte),3.600,448,Anuncio actualizado el 21/10/2024,N/A,N/A,"[(Superficie construida:, 448 m²), (Habitacion...",Alquiler,2024-10-30T18:17:23.987867,7b417268-cd0f-433c-b2ec-29ad7c8fd2e7
5,Piso en alquiler en Calle de Vista Alegre,Somosaguas-Húmera-Los Ángeles (Pozuelo de Alar...,1.450,69,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 69 m²), (Superficie ...",Alquiler,2024-10-30T18:17:30.717978,e36e0cee-d529-4433-a99a-de6434560c92
6,Piso en alquiler en Valdemarín,Valdemarín (Distrito Moncloa-Aravaca. Madrid C...,2.200,70,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 70 m²), (Habitacione...",Alquiler,2024-10-30T18:17:37.166850,f557289f-57bd-412c-a079-456ae412414d
7,Ático en alquiler en Calle de Felipe IV,Jerónimos (Distrito Retiro. Madrid Capital),4.500,133,Anuncio actualizado el 22/10/2024,N/A,N/A,"[(Superficie construida:, 133 m²), (Superficie...",Alquiler,2024-10-30T18:17:43.579989,717af916-8044-431a-88d1-a2102b08b913
8,Piso en alquiler en Calle de Irún,Casa de Campo (Distrito Moncloa-Aravaca. Madri...,3.000,85,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 85 m²), (Habitacione...",Alquiler,2024-10-30T18:17:51.301557,4c524b91-efa0-4077-9e73-7ddbde6525db
9,Piso en alquiler en Calle de Cea Bermúdez,Vallehermoso (Distrito Chamberí. Madrid Capital),2.400,100,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 100 m²), (Habitacion...",Alquiler,2024-10-30T18:17:57.825348,61ce9c9c-29d3-4059-8da0-397c001c9386


In [8]:
cantidad_unicos = df["id"].nunique()
print(cantidad_unicos)


50


- A continuación procedemos de la misma forma pero con la compra

In [4]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from datetime import datetime
import uuid

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.67 Safari/537.36"
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/venta/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except:
    pass  # No mostramos mensaje si el botón de cookies no está presente

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except:
    browser.quit()

# Hacer clic en la primera caja de la lista de inmuebles
try:
    primer_inmueble = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.ad-preview'))
    )
    primer_inmueble.click()
    time.sleep(random.uniform(10, 12))  # Espera para evitar detección
except:
    browser.quit()

# Lista para almacenar los datos
all_data = []
registro_count = 0  # Contador de registros
max_registros = 50  # Límite de registros a procesar

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while registro_count < max_registros:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')
        url_actual = browser.current_url

        # Intentar extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

        except:
            # Si falla la extracción de datos, asigna valores "N/A" pero sigue contabilizando el registro
            precio = "N/A"
            superficie_construida = "N/A"
            ultima_actualizacion = "N/A"
            features_list = []
            consumo = "N/A"
            emisiones = "N/A"

        # Generar un timestamp de scrapeo y un identificador único
        timestamp_scrapeo = datetime.now().isoformat()
        unique_id = str(uuid.uuid4())

        # Guarda los datos en una lista, incluyendo el timestamp y el ID único
        data = {
            'ID': unique_id,
            'Timestamp Scrapeo': timestamp_scrapeo,
            'Enlace': url_actual,
            'Precio': precio,
            'Superficie Construida': superficie_construida,
            'Última Actualización': ultima_actualizacion,
            'Consumo Energético': consumo,
            'Emisiones CO2': emisiones,
            'Características': features_list,
            'Tipo de operación': 'Compra'
        }

        all_data.append(data)
        registro_count += 1  # Incrementar el contador de registros

        # Imprime los datos de la página actual, incluso si hay errores
        print(f"Inmueble {registro_count}: {data}")

        # Pausar cada 400 registros
        if registro_count % 400 == 0:
            pausa = random.uniform(45, 60)
            print(f"Pausa de {pausa:.2f} segundos tras {registro_count} registros.")
            time.sleep(pausa)

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation-arrow--next"))
            )
            siguiente_boton.click()

        except:
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except:
        break  # Ignorar errores al cargar la página del inmueble y continuar con el siguiente

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df1 = pd.DataFrame(all_data)
print(df1)

# Opcional: guarda el DataFrame en un archivo CSV
df1.to_csv('muestra_compra_50_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'ID': '92ac36eb-5b4b-4187-80a0-dbbef9ebd5a1', 'Timestamp Scrapeo': '2024-10-30T18:33:25.916519', 'Enlace': 'https://www.pisos.com/comprar/chalet_pareado-serranillos_del_valle_centro_urbano-45091667204_993816/', 'Precio': '374.000', 'Superficie Construida': '264', 'Última Actualización': 'Anuncio actualizado el 30/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '264 m²'), ('Superficie útil:', '264 m²'), ('Superficie solar:', '457 m²'), ('Habitaciones:', '5'), ('Baños:', '4'), ('Antigüedad:', 'Entre 20 y 30 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '4993816-003274')], 'Tipo de operación': 'Compra'}
Inmueble 2: {'ID': '9c5aaef2-c4cd-429c-8a0d-286cc7d8dbfc', 'Timestamp Scrapeo': '2024-10-30T18:33:33.379541', 'Enlace': 'https://www.pisos.com/comprar/piso-el_guijo_colonia_espana28260-46688539793_510261/', 'Precio': 'N/A', 'Superficie Construida': 'N/A', 'Última Actualización': 'N/A', 'Consumo Energé

In [5]:
df1

,ID,Timestamp Scrapeo,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación
0,92ac36eb-5b4b-4187-80a0-dbbef9ebd5a1,2024-10-30T18:33:25.916519,https://www.pisos.com/comprar/chalet_pareado-s...,374.000,264,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 264 m²), (Superficie...",Compra
1,9c5aaef2-c4cd-429c-8a0d-286cc7d8dbfc,2024-10-30T18:33:33.379541,https://www.pisos.com/comprar/piso-el_guijo_co...,N/A,N/A,N/A,N/A,N/A,[],Compra
2,ae91be23-38b6-4dec-95a4-b5d1e6861b3d,2024-10-30T18:33:45.453993,https://www.pisos.com/comprar/atico-nino_jesus...,N/A,N/A,N/A,N/A,N/A,[],Compra
3,183b8bb0-faf0-4478-9951-805deeb43607,2024-10-30T18:33:49.933647,https://www.pisos.com/comprar/casa_adosada-sev...,299.900,186,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 186 m²), (Superficie...",Compra
4,0341d74a-50f1-4b06-a010-82ecbbcd536f,2024-10-30T18:33:57.449281,https://www.pisos.com/comprar/casa_adosada-cas...,540.000,211,Anuncio actualizado el 04/07/2024,N/A,N/A,"[(Superficie construida:, 211 m²), (Habitacion...",Compra
5,229d602e-1e43-49ca-a71f-64f0571f8cbd,2024-10-30T18:34:02.542208,https://www.pisos.com/comprar/duplex-galapagar...,N/A,N/A,N/A,N/A,N/A,[],Compra
6,9dfede36-1cd2-4482-af32-5ba3ddc75fe5,2024-10-30T18:34:07.726029,https://www.pisos.com/comprar/duplex-centro_la...,279.700,107,Anuncio actualizado el 30/10/2024,N/A,N/A,"[(Superficie construida:, 107 m²), (Habitacion...",Compra
7,4967c5e4-cc07-47d3-8750-1a852a3d3aec,2024-10-30T18:34:11.719949,https://www.pisos.com/comprar/chalet-los_penas...,695.000,260,Anuncio actualizado el 27/09/2024,N/A,N/A,"[(Superficie construida:, 260 m²), (Superficie...",Compra
8,dcb1964d-e9df-4b83-82b9-59881aaf1ce7,2024-10-30T18:34:16.059216,https://www.pisos.com/comprar/piso-zofio28026-...,N/A,N/A,N/A,N/A,N/A,[],Compra
9,311258ef-1858-4388-ac1e-4200d65849fc,2024-10-30T18:34:20.307418,https://www.pisos.com/comprar/casa-chinchon_ce...,230.000,234,Anuncio actualizado el 24/10/2024,N/A,N/A,"[(Superficie construida:, 234 m²), (Superficie...",Compra


In [6]:
cantidad_unicos = df1["ID"].nunique()
print(cantidad_unicos)


50
